In [17]:
import re
import operator

from glob import glob
import pandas as pd

In [18]:
filenames = glob('data/full_who_dataset*.csv')
dataframes = [pd.read_csv(open(f, 'rb')) for f in filenames]
data = pd.concat(dataframes)

In [19]:
data = data.drop(columns=['Unnamed: 0', 'level_0', 'index', ' '])
data.reset_index(drop=True, inplace=True)

Extract tweets from/about organisations from list of organisations

In [20]:
data_a11 = pd.read_csv('data/org/a11.tsv', delimiter='\t')
data_a12 = pd.read_csv('data/org/a12.tsv', delimiter='\t')

In [23]:
def org_extraction(dataframe):
    for x in range(len(dataframe)):
        org = []
        org_mention = dataframe.loc[x]['account']
        org_account = re.sub('@', '', org_mention)
        org_local_name = dataframe.loc[x]['local_name']
        org_en_name = dataframe.loc[x]['english_name']

        for num, row in enumerate(data['text']):
            if type(org_mention) == str:
                if re.findall(org_mention, str(row)):
                    org.append(data.loc[num])
            elif type(org_local_name) == str:
                if re.findall(org_local_name, str(row)):
                    org.append(data.loc[num])
            elif type(org_en_name) == str:
                if re.findall(org_en_name, str(row)):
                    org.append(data.loc[num])
        
        for num, row in enumerate(data['from_user']):
            if org_account == row:
                org.append(data.loc[num])
                
        
        org_data = pd.DataFrame(org)
        org_data.drop_duplicates(subset ="id_str", keep = False, inplace = True)
        org_data.drop_duplicates(subset ="text", keep = False, inplace = True)
        org_data = org_data.reset_index(drop=True)
        org_data.to_csv('data/out/a11_'+ org_account +'.csv', sep='\t')

In [24]:
org_extraction(data_a11)

Find top-50 most frequent messages from the datasets

In [11]:
top50 = {}

for text in data['text']:
    if text not in top50:
        top50[text] = 1
    else:
        top50[text] += 1

top50_freq = sorted(top50.items(), key=operator.itemgetter(1), reverse=True)